In [1]:
import os 
import pandas as pd 

def loadCSVs(path):
    _dfs = []
    for file in os.listdir(path):
        _dict = {}
        if file.endswith('.csv'):
            full_path = os.path.join(path, file)
            df = pd.read_csv(full_path)
            _dict[file] = df
            _dfs.append(_dict)

    return _dfs


In [2]:
COLUMNS = 'columns'
DATA_TYPE = 'data_type'
DATA = 'data'
METRICS = 'metrics'
OG_FILE = 'original_file_name'
PLAYER_TYPE = 'player_type'

In [3]:
root = "c:\\"
path = root + 'fangraphs'
dfs = loadCSVs(path)
import intake

_json_list = []
for _dict in dfs: 
    for file_name, df in _dict.items():
        data_json = {}
        file_name = file_name.split('.')[0] # fangraphs-advanced.csv -> fangraphs-advanced
        num_parts = len(file_name.split('-'))
        player_type = 'pitcher' if 'pitcher' in file_name else 'batter'
        data_json[PLAYER_TYPE] = player_type
        data_json[DATA_TYPE] = 'sabermetric' if num_parts > 3 else 'pitch_level' 
        data_json[METRICS] = file_name.split(player_type)[-1]
        data_json[DATA] = df
        data_json[COLUMNS] = list(df.columns)
        data_json[OG_FILE] = file_name

        _json_list.append(data_json)

In [4]:
metric_scrubber = intake.MetricScrub(_json_list)

In [5]:
map = metric_scrubber.standardizeColumnNames()
for _dict in _json_list:
    _dict[DATA] = _dict[DATA].rename(columns=map).to_json(orient='records')

In [6]:
import json
with open('fangraphs.json', 'w') as file:
    json.dump(_json_list, file, indent=4)